In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import requests
import json

In [2]:
main_url = 'https://kw-aruba.com/'
sub_links = ["listings/for-sale/residential", "listings/for-sale/condominium-townhouse",
             "listings/for-sale/land", "listings/for-sale/commercial", "listings/for-rent/residential",
             "listings/for-rent/condominium-townhouse", "listings/for-rent/land", "listings/for-sale/commercial"]

In [7]:
def join_links(main, sub):
    links  = []
    for sub in sub_links:
        url = main_url + sub
        links.append(url)
    return links
links = join_links(main_url, sub_links)

In [8]:
links

['https://kw-aruba.com/listings/for-sale/residential',
 'https://kw-aruba.com/listings/for-sale/condominium-townhouse',
 'https://kw-aruba.com/listings/for-sale/land',
 'https://kw-aruba.com/listings/for-sale/commercial',
 'https://kw-aruba.com/listings/for-rent/residential',
 'https://kw-aruba.com/listings/for-rent/condominium-townhouse',
 'https://kw-aruba.com/listings/for-rent/land',
 'https://kw-aruba.com/listings/for-sale/commercial']

In [18]:
def get_all_url(li):
    all_links = []
    for link in li:
        req = requests.get(link)
        soup = BeautifulSoup(req.content, "html.parser")
        anchors = soup.find('div', {'class': 'properties-grid'})
        for anchor in anchors:
            links  = anchor.find_all('a', href = True)
            for link in links:
                urls = main_url+ link.get('href')
                #print(urls)
                all_links.append(urls)
    return all_links
path = get_all_url(links)

https://kw-aruba.com//listings/l-g-smith-blvd
https://kw-aruba.com//listings/opal-125
https://kw-aruba.com//listings/westpunt-12-c
https://kw-aruba.com//listings/AltoVista45A
https://kw-aruba.com//listings/seroe-janchi-31
https://kw-aruba.com//listings/opal-ocean-view
https://kw-aruba.com//listings/altovista-37-g
https://kw-aruba.com//listings/opal-133
https://kw-aruba.com//listings/52-tierra-del-sol
https://kw-aruba.com//listings/koyari-apartments-2
https://kw-aruba.com//listings/villa-esmeralda
https://kw-aruba.com//listings/safir-39
https://kw-aruba.com//listings/villa-westpunt
https://kw-aruba.com//listings/paradera-104-c
https://kw-aruba.com//listings/cfj4
https://kw-aruba.com//listings/Villa-Casa-Blanca
https://kw-aruba.com//listings/tuscany-villas
https://kw-aruba.com//listings/seroe-blanco-56-l
https://kw-aruba.com//listings/seroe-blanco-54-
https://kw-aruba.com//listings/palm-beach-33-
https://kw-aruba.com//listings/alto-vista-villa-park
https://kw-aruba.com//listings/ponton-3

In [12]:
def get_details(urls):
    property_list = []
    for link in urls:
        req = requests.get(link)
        soup = BeautifulSoup(req.content, "html.parser")
        name = soup.find('span', class_ = "headline__inner").text
        location = soup.find('div', class_ = "page-property-details__location").text
        property_status = soup.find('div', class_ = "page-property-details__type").text
        price = soup.find('div', class_ = "page-property-details__price").text.strip()
        rooms = soup.find("td", text="Bedrooms").find_next_sibling("td").text
        full_bathrooms = soup.find("td", text="Full bathrooms").find_next_sibling("td").text
        half_bathrooms = soup.find("td", text="Half bathrooms").find_next_sibling("td").text
        prop = {
            'name': name,
            'location': rating,
            'property': reviews,
            'price': price,
            'rooms': rooms,
            'bathrooms' : full_bathrooms
        }
        property_list.append(prop)
        print('Saving:', prop['name'])
        
    df = pd.DataFrame(property_list)
    print(df.head(20))
    df.to_csv('kw_aruba.csv', header=True, index=False)

In [13]:
get_details(path)

TypeError: 'NoneType' object is not iterable

In [17]:
property_list = []
for link in path:
    req = requests.get(link)
    soup = BeautifulSoup(req.content, "html.parser")
    name = soup.find('span', class_ = "headline__inner").text
    location = soup.find('div', class_ = "page-property-details__location").text
    property_status = soup.find('div', class_ = "page-property-details__type").text
    price = soup.find('div', class_ = "page-property-details__price").text.strip()
    rooms = soup.find("td", text="Bedrooms").find_next_sibling("td").text
    full_bathrooms = soup.find("td", text="Full bathrooms").find_next_sibling("td").text
    half_bathrooms = soup.find("td", text="Half bathrooms").find_next_sibling("td").text
    prop = {
            'name': name,
            'location': rating,
            'property': reviews,
            'price': price,
            'rooms': rooms,
            'bathrooms' : full_bathrooms
        }
    property_list.append(prop)
    print('Saving:', prop['name'])

TypeError: 'NoneType' object is not iterable